# Prerrequisites

Installing Spark

---



In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

Starting Spark Session and print the version


---


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .getOrCreate()

spark.version

Creating tunnel</br>
**To Check the Spark UI, open the URL printed by running the above command : https://######/jobs/, /SQL/**


In [ ]:
 from google.colab.output import eval_js
 print(eval_js("google.colab.kernel.proxyPort(4040)") + "jobs/")

# Download Datasets

In [ ]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/netflix_titles.csv -P /dataset
!ls /dataset

# Reading Data with Spark SQL

---



## Example 1

In [ ]:
!head /dataset/bank.csv

Converting a RDD to a DataFrame

In [ ]:
from pyspark.sql.types import Row
from pyspark.sql.functions import *

bankText = spark.sparkContext.textFile("/dataset/bank.csv")

bank = bankText.map(lambda lineaCsv: lineaCsv.split(";"))\
.filter(lambda s: s[0] != "\"age\"") \
.map(lambda row: Row(int(row[0]), row[1].replace("\"", ""), row[2].replace("\"", ""), row[3].replace("\"", ""), row[5].replace("\"", ""))) \
.toDF(["age", "job", "marital", "education", "balance"]) \
.withColumn("age", col("age").cast("int"))

In [ ]:
bank.printSchema()

In [ ]:
bank.createOrReplaceTempView("bank")

Loading a **Google Colab extension** to show a table with filters

In [ ]:
%load_ext google.colab.data_table

In [ ]:

from pyspark.sql.functions import *

bank_grouped = bank\
.groupBy(bank.marital) \
.agg({"balance": "avg"}) \
.select("marital", col("avg(balance)").alias("balance_avg")) \
.orderBy(col("balance_avg").desc())\

bank_grouped.show()


In [ ]:
bank_grouped.toPandas()

In [ ]:
spark.sql("SELECT marital, avg(balance) as balance_avg FROM bank group by marital").show()

In [ ]:
import plotly.express as px

fig = px.pie(bank_grouped.toPandas(), values='balance_avg', names='marital', title='By Marital')
fig.show()

## Example 2

Loading a CSV file as RDD, converting into a DataFrame, applying a specific schema using the method `createDataFrame`

In [ ]:
from pyspark.sql.types import *

bankSchema = StructType([
    StructField("age", IntegerType(), False), 
    StructField("job", StringType(), False),
    StructField("marital", StringType(), False),
    StructField("education", StringType(), False),
    StructField("balance", IntegerType(), False)])

bankText = spark.sparkContext.textFile("/dataset/bank.csv")

bank = bankText\
.map(lambda s: s.split(";")).filter(lambda s: s[0] != "\"age\"")\
.map(lambda s:(int(s[0]), str(s[1]).replace("\"", ""), str(s[2]).replace("\"", ""), str(s[3]).replace("\"", ""), int(s[5]) ))

bankdf = spark.createDataFrame(bank, bankSchema)
bankdf.createOrReplaceTempView("bank2")

In [ ]:
spark.sql("select * from bank2 limit 10").show()

## Exercise 1
Load file `vehicles.csv` to a DataFrame, showing the content and printing the schema.

Use this [documentation](https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html) to read data in a DataFrame

---



Filter out the previous DafaFrame to get vehicles where the capicity is greater than 70

---



# Spark SQL. Aggregation Functions

Useful Links:

http://spark.apache.org/docs/latest/api/python/


## Exercise 2

Using the DataFrame with all vehicles loaded in Exercise 1, get the number of passengers by vehicle class


---




## Exercise 3

Load the file `characters.csv` getting the most common eye color among all characters

---

## Exercise 4

1. Load characters DataFrame into a temporary table
2. Using SQL, get the number of characters by gender


---



## Exercise 5

Load `netflix_titles.csv` file in a DataFrame, printing the schema

---



In [ ]:
!head /dataset/netflix_titles.csv

## Exercise 6

Get the year in which most films were added(No TV Shows). Use a UDF to get the year

---

